In [1]:
!which python

/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/bin/python


In [2]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [3]:
# Model from Hugging Face hub
base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
# New instruction dataset
stack_overflow_dataset = "Mxode/StackOverflow-QA-C-Language-40k"
code_feedback_dataset = "m-a-p/CodeFeedback-Filtered-Instruction"

# Fine-tuned model
new_model = "llama-2-7b-chat-stack-overflow"

In [4]:
dataset = load_dataset(code_feedback_dataset, split="train")
dataset = dataset.shuffle(seed=42)

In [5]:
# dataset currently has 'question' and 'answer' columns. We need to combine them into a single column named 'text'
dataset = dataset.map(
    lambda examples: {
        "text": "<s>[INST]" + examples["query"] + "[\INST] " + examples["answer"] + "</s>",
    }
)
# Select only 40000 examples
trunc_dataset = dataset.select(range(40000))
del dataset

In [6]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [9]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [10]:
training_params = TrainingArguments(
    output_dir="./tiny-llama-code-feedback",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [11]:
trainer = SFTTrainer(
    model=model,
    train_dataset=trunc_dataset,
    peft_config=peft_params,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/home/bgsmagnuson/Documents/code_py/LLM-Debug-Assistant/venv/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [12]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

  0%|          | 0/10000 [00:00<?, ?it/s]

{'loss': 1.1039, 'grad_norm': 0.1473865509033203, 'learning_rate': 0.0002, 'epoch': 0.0}
{'loss': 1.1087, 'grad_norm': nan, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 1.1259, 'grad_norm': 0.20463761687278748, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 1.08, 'grad_norm': 2.0228610038757324, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 1.0137, 'grad_norm': 0.2088489979505539, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 1.0303, 'grad_norm': 1.0524128675460815, 'learning_rate': 0.0002, 'epoch': 0.01}
{'loss': 1.0345, 'grad_norm': 0.16082286834716797, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 1.0448, 'grad_norm': 0.7031107544898987, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 0.9979, 'grad_norm': 0.19572685658931732, 'learning_rate': 0.0002, 'epoch': 0.02}
{'loss': 1.0275, 'grad_norm': 0.9878652691841125, 'learning_rate': 0.0002, 'epoch': 0.03}
{'loss': 0.9301, 'grad_norm': 0.16263549029827118, 'learning_rate': 0.0002, 'epoch': 0.03}
{'loss': 1.1053, 'grad_n

In [14]:
trainer.push_to_hub("tiny-llama-code-feedback")

Upload 8 LFS files:   0%|          | 0/8 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/36.1M [00:00<?, ?B/s]

events.out.tfevents.1713414465.helios.8602.0:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

events.out.tfevents.1713414705.helios.9028.0:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

events.out.tfevents.1713414787.helios.9547.0:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

events.out.tfevents.1713414905.helios.5100.0:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

events.out.tfevents.1713414939.helios.5551.0:   0%|          | 0.00/5.15k [00:00<?, ?B/s]

events.out.tfevents.1713415047.helios.5749.0:   0%|          | 0.00/89.9k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/bgsmagnuson/tiny-llama-code-feedback/commit/fa28f0626aa8b0c1bdc1dbdbd217b352a8cf1c8d', commit_message='tiny-llama-code-feedback', commit_description='', oid='fa28f0626aa8b0c1bdc1dbdbd217b352a8cf1c8d', pr_url=None, pr_revision=None, pr_num=None)